# Load Library

In [1]:
#file read, 환경변수, 정규식
import glob,os, re
from dotenv import load_dotenv
from tqdm import tqdm
from typing import List, Any

In [3]:
#!pip install pdfplumber
#!pip install langchain_text_splitters
#!pip install langchain_chroma
#!pip install langchain_ibm
# PDF 처리 및 텍스트 청킹
import pdfplumber
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 벡터 저장소 및 임베딩 모델
from langchain_chroma import Chroma
from langchain_ibm import WatsonxEmbeddings

In [4]:
#langchain chain
from langchain_core.runnables import RunnablePassthrough, RunnableConfig
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import ChatWatsonx, WatsonxLLM

In [5]:
# 환경변수 로드(.env 파일 참고)
load_dotenv(verbose=True)

True

# 전자제품 사용설명서 처리

# 전처리 코드

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
pdf_path="./data"

In [8]:
def loadpreProcess(pdf_path:str, text_splitter:RecursiveCharacterTextSplitter) -> List[Document]:
    """경로 내 모든 PDF 파일 로드 및 청킹.

    Args:
        pdf_path (str): PDF 파일이 들어있는 경로
        text_splitter (RecursiveCharacterTextSplitter): 텍스트 스플리터

    Returns:
        List[Document]: 청킹 완료된 모든 문서(metadatd 포함)
    """
    pdf_files  = glob.glob(os.path.join(pdf_path, '*.pdf'))
    pattern = r"\\(.*?)\.pdf"
    cellphone_docu=[]
    for pdf_file in tqdm(pdf_files, desc="PDF 파일 로드 중 :"):
        product_name = re.findall(pattern, pdf_file)[0]
        documents_page=[]
        pdf = pdfplumber.open(pdf_file)
        pages = pdf.pages
        for page in pages:
            page_index=""
            page_number=""
            page_text=""
            for texts in page.chars:
                if texts.get('y0')>=806:
                    #상단 index 추출
                    page_index+=texts.get('text')
                elif texts.get('y1')<=28:
                    #상단 index 추출
                    page_number+=texts.get('text')
                else:
                    page_text+=texts.get('text')
            page_docu=Document(
            page_content=page_text,
            metadata={"index":page_index,
                    "product_name":product_name,
                    "page":page_number}
            )
            documents_page.append(page_docu)
        spilt_doc=text_splitter.split_documents(documents_page)
        cellphone_docu+=spilt_doc
    return cellphone_docu

In [9]:
cellphone_docu=loadpreProcess(pdf_path, text_splitter)

PDF 파일 로드 중 :: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:34<00:00,  8.66s/it]


In [24]:
#for i in cellphone_docu:
#    print(i)

# Embedding Model & VectorDB

In [10]:
EMBED_PARAMS = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 512,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

In [11]:
watsonx_embedding = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    url=os.getenv('IBM_CLOUD_URL'),
    project_id=os.getenv('PROJECT_ID'),
    apikey="t9oPtc-l3HMQH54UFeLvRfh5s_fnOg2az_C7Ss7mDiVT",
    params=EMBED_PARAMS,
)

In [12]:
chroma_db=Chroma(embedding_function=watsonx_embedding, 
                 collection_name='manual_vector_db', 
                 persist_directory='./db/manual/chromadb_0',
                 collection_metadata = {'hnsw:space': 'cosine'})

⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [13]:
def storeToDBInBatchs(db:Any, docs:List[Document], batch_size:int, reset_collection:bool):
    """커널 충돌 방지용 배치단위 DB저장(ChromaDB)

    Args:
        db (Any): ChromaDB
        docs (List[Document]): 저장할 메뉴얼 문서 list
        batch_size (int): 배치 사이즈
        reset_collection (bool): 저장 전 컬렉션 리셋 여부
    """
    if reset_collection:
        db.reset_collection() #컬렉션 리셋
        
    for i in tqdm(range(0,len(docs),batch_size), desc="배치단위 메뉴얼 문서 저장 중 :"):
        batch = docs[i:i+batch_size]
        db.add_documents(batch)

In [14]:
storeToDBInBatchs(chroma_db, cellphone_docu, 1000, True)

배치단위 메뉴얼 문서 저장 중 :: 100%|████████████████████████████████████████████████████| 2/2 [00:45<00:00, 22.72s/it]


In [15]:
mmr_retriever=chroma_db.as_retriever(
    search_type='similarity',
    search_kwargs={"k":5, 'filter':{"product_name":"Z플립",}})

# LLM Model

In [16]:
LLM_PARAMS = {
    "decoding_method": "sample",
    "temperature": 0.99,
    "top_k": 20,
    "top_p": 0.5,
    'repetition_penalty':1,
    "max_new_tokens": 1000,
    "min_new_tokens": 1,
    "stop_sequences": ['<|eot_id|>', '<|end_of_text|>'],
}

In [17]:
chat = ChatWatsonx(
    model_id='meta-llama/llama-3-1-8b-instruct',
    url=os.getenv('IBM_CLOUD_URL'),
    project_id=os.getenv('PROJECT_ID'),
    apikey="t9oPtc-l3HMQH54UFeLvRfh5s_fnOg2az_C7Ss7mDiVT",
    params=LLM_PARAMS,
)

In [18]:
prompt=PromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
당신은 주어진 Context를 근거로 Question에 답변하는 assistant입니다.
당신은 친절하고 사용자에게 도움이 되기 위해 성실히 응답은 해야합니다.
아래 조건에 맞춰 답변해주세요.

조건 :
- User의 Context를 기반하여 답변하세요.
- 해당 내용이 User의 Context에 없다면 "현재 매뉴얼로 알 수 없습니다."라고 말해줘.
- 답변은 최대 5문장 이내로 답변해주세요.
<|eot_id|><|start_header_id|>user<|end_header_id|> 

Question: {question} 

Context: {context} 

<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
)

In [19]:
rag_chain = (
    {"context": mmr_retriever, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

In [20]:
print(rag_chain.invoke('오산에서 가까운 서비스 센터는'))

 현재 매뉴얼로 알 수 없습니다.
